<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkAdaptiveQueryExec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('SparkAQE').getOrCreate()

In [2]:
spark

In [3]:
transactions=[
    (4, 'Apples', 1.03),
    (3, 'Apples', 2.68),
    (4, 'Oranges', 4.72),
    (1, 'Apples', 1.94),
    (3, 'Bread', 3.07),
    (1, 'Apples', 0.75),
    (4, 'Bananas', 4.71),
    (2, 'Bread', 2.86),
    (3, 'Bread', 2.61),
    (3, 'Apples', 0.64),
    (1, 'Oranges', 1.60),
    (1, 'Bread', 1.50)
]

schema=['store_id', 'product', 'price']
df1=spark.createDataFrame(data=transactions, schema=schema)
df1.show()


+--------+-------+-----+
|store_id|product|price|
+--------+-------+-----+
|       4| Apples| 1.03|
|       3| Apples| 2.68|
|       4|Oranges| 4.72|
|       1| Apples| 1.94|
|       3|  Bread| 3.07|
|       1| Apples| 0.75|
|       4|Bananas| 4.71|
|       2|  Bread| 2.86|
|       3|  Bread| 2.61|
|       3| Apples| 0.64|
|       1|Oranges|  1.6|
|       1|  Bread|  1.5|
+--------+-------+-----+



In [4]:
store=[
    (1, "New York"),
    (2, "Los Angeles"),
    (3, "Chicago"),
    (4, "Houston")
]
schema1=['store_id', 'store_name']
df2=spark.createDataFrame(data=store, schema=schema1)
df2.show()

+--------+-----------+
|store_id| store_name|
+--------+-----------+
|       1|   New York|
|       2|Los Angeles|
|       3|    Chicago|
|       4|    Houston|
+--------+-----------+



With AQE disabled

In [5]:
spark.conf.set("spark.sql.adaptive.enabled", "False")

In [6]:
df3=df1.join(df2, on='Store_id', how='inner')
df3.show()

+--------+-------+-----+-----------+
|store_id|product|price| store_name|
+--------+-------+-----+-----------+
|       1| Apples| 1.94|   New York|
|       1| Apples| 0.75|   New York|
|       1|Oranges|  1.6|   New York|
|       1|  Bread|  1.5|   New York|
|       3| Apples| 2.68|    Chicago|
|       3|  Bread| 3.07|    Chicago|
|       3|  Bread| 2.61|    Chicago|
|       3| Apples| 0.64|    Chicago|
|       2|  Bread| 2.86|Los Angeles|
|       4| Apples| 1.03|    Houston|
|       4|Oranges| 4.72|    Houston|
|       4|Bananas| 4.71|    Houston|
+--------+-------+-----+-----------+



In [7]:
df3.explain()

== Physical Plan ==
*(5) Project [store_id#0L, product#1, price#2, store_name#20]
+- *(5) SortMergeJoin [store_id#0L], [store_id#19L], Inner
   :- *(2) Sort [store_id#0L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(store_id#0L, 200), ENSURE_REQUIREMENTS, [plan_id=117]
   :     +- *(1) Filter isnotnull(store_id#0L)
   :        +- *(1) Scan ExistingRDD[store_id#0L,product#1,price#2]
   +- *(4) Sort [store_id#19L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(store_id#19L, 200), ENSURE_REQUIREMENTS, [plan_id=123]
         +- *(3) Filter isnotnull(store_id#19L)
            +- *(3) Scan ExistingRDD[store_id#19L,store_name#20]




With AQE enabled

In [8]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

In [9]:
df4=df1.join(df2, on='store_id', how='inner')
df4.show()

+--------+-------+-----+-----------+
|store_id|product|price| store_name|
+--------+-------+-----+-----------+
|       1| Apples| 1.94|   New York|
|       1| Apples| 0.75|   New York|
|       1|Oranges|  1.6|   New York|
|       1|  Bread|  1.5|   New York|
|       2|  Bread| 2.86|Los Angeles|
|       3| Apples| 2.68|    Chicago|
|       3|  Bread| 3.07|    Chicago|
|       3|  Bread| 2.61|    Chicago|
|       3| Apples| 0.64|    Chicago|
|       4| Apples| 1.03|    Houston|
|       4|Oranges| 4.72|    Houston|
|       4|Bananas| 4.71|    Houston|
+--------+-------+-----+-----------+



In [10]:
df4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [store_id#0L, product#1, price#2, store_name#20]
   +- SortMergeJoin [store_id#0L], [store_id#19L], Inner
      :- Sort [store_id#0L ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(store_id#0L, 200), ENSURE_REQUIREMENTS, [plan_id=283]
      :     +- Filter isnotnull(store_id#0L)
      :        +- Scan ExistingRDD[store_id#0L,product#1,price#2]
      +- Sort [store_id#19L ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(store_id#19L, 200), ENSURE_REQUIREMENTS, [plan_id=284]
            +- Filter isnotnull(store_id#19L)
               +- Scan ExistingRDD[store_id#19L,store_name#20]




In this Case, there might not be a major difference, but when there are 2 large dataframe joined, or a query is executed, there a difference can be seen in  shuffling parameters, while joining the type of join used, within the spark app.

**END OF CODE**